In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  
        self.fc2 = nn.Linear(hidden_size, int(hidden_size / 2))
        self.dropout2 = nn.Dropout(dropout_rate)  
        self.fc3 = nn.Linear(int(hidden_size / 2), output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        # out = self.dropout1(out)  
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.dropout2(out)  
        out = self.fc3(out)
        return out
    
input_size = 365*19
hidden_size = 1024
output_size = 1  
model = MLP(input_size, hidden_size, output_size)
model.to('cuda')
train_dataset = torch.load('../../../data/cleaned/train.pt')
train_x, train_y = train_dataset.tensors  

val_dataset = torch.load('../../../data/cleaned/val.pt')
test_x, test_y = val_dataset.tensors  

train_x = train_x.reshape(-1, 365*19)
test_x = test_x.reshape(-1, 365*19)

train_x = train_x.to('cuda')
test_x = test_x.to('cuda')
train_y = train_y.to('cuda')
test_y = test_y.to('cuda')

print(train_x.shape)
criterion = nn.MSELoss()  

optimizer = optim.Adam(model.parameters(), lr=5e-5)
num_epochs = 1500

for epoch in range(num_epochs):
    model.train()  
    outputs = model(train_x)
    loss = criterion(outputs, train_y.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        model.eval()  
        with torch.no_grad():
            val_outputs = model(test_x)
            val_loss = criterion(val_outputs, test_y.unsqueeze(1))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.sqrt(loss.item()):.4f}, Validation Loss: {np.sqrt(val_loss.item()):.4f}')

torch.Size([85244, 6935])
Epoch [100/1500], Loss: 281.4065, Validation Loss: 288.6800
Epoch [200/1500], Loss: 277.6067, Validation Loss: 286.4980
Epoch [300/1500], Loss: 270.6946, Validation Loss: 283.6929
Epoch [400/1500], Loss: 255.7054, Validation Loss: 272.9358
Epoch [500/1500], Loss: 223.4408, Validation Loss: 252.6665
Epoch [600/1500], Loss: 171.7375, Validation Loss: 220.6344
Epoch [700/1500], Loss: 121.8603, Validation Loss: 191.5247
Epoch [800/1500], Loss: 87.1191, Validation Loss: 176.1667
Epoch [900/1500], Loss: 67.8100, Validation Loss: 168.7058
Epoch [1000/1500], Loss: 57.2283, Validation Loss: 164.7003
Epoch [1100/1500], Loss: 51.2472, Validation Loss: 162.8170
Epoch [1200/1500], Loss: 47.1411, Validation Loss: 161.9268
Epoch [1300/1500], Loss: 43.7308, Validation Loss: 161.4277
Epoch [1400/1500], Loss: 40.6295, Validation Loss: 161.1944
Epoch [1500/1500], Loss: 37.7138, Validation Loss: 161.0667
